In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv("./datasets/ml-25m/movies.csv")

In [3]:
ratings = pd.read_csv("./datasets/ml-25m/ratings.csv")

In [3]:
import tensorflow

In [5]:
import os
import sys
from tempfile import TemporaryDirectory

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources, prepare_hparams
from recommenders.models.deeprec.models.xDeepFM import XDeepFMModel
from recommenders.models.deeprec.io.iterator import FFMTextIterator

print(f"System version: {sys.version}")

System version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]


In [ ]:

from recommenders.utils.notebook_utils import store_metadata

In [5]:
import torch

In [6]:
EPOCHS = 10
BATCH_SIZE = 4096
RANDOM_SEED = 42  # Set this to None for non-deterministic result

In [7]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name
yaml_file = os.path.join(data_path, r'xDeepFM.yaml')
output_file = os.path.join(data_path, r'output.txt')
train_file = os.path.join(data_path, r'cretio_tiny_train')
valid_file = os.path.join(data_path, r'cretio_tiny_valid')
test_file = os.path.join(data_path, r'cretio_tiny_test')

if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/deeprec/', data_path, 'xdeepfmresources.zip')

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 10.3k/10.3k [00:13<00:00, 755KB/s]


In [8]:
print('Demo with Criteo dataset')
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=2300000, 
                          FIELD_COUNT=39, 
                          cross_l2=0.01, 
                          embed_l2=0.01, 
                          layer_l2=0.01,
                          learning_rate=0.002, 
                          batch_size=BATCH_SIZE, 
                          epochs=EPOCHS, 
                          cross_layer_sizes=[20, 10], 
                          init_value=0.1, 
                          layer_sizes=[20,20],
                          use_Linear_part=True, 
                          use_CIN_part=True, 
                          use_DNN_part=True)
print(hparams)

Demo with Criteo dataset
HParams object with values {'use_entity': True, 'use_context': True, 'cross_activation': 'identity', 'user_dropout': False, 'dropout': [0.0, 0.0], 'attention_dropout': 0.0, 'load_saved_model': False, 'fast_CIN_d': 0, 'use_Linear_part': True, 'use_FM_part': False, 'use_CIN_part': True, 'use_DNN_part': True, 'init_method': 'tnormal', 'init_value': 0.1, 'embed_l2': 0.01, 'embed_l1': 0.0, 'layer_l2': 0.01, 'layer_l1': 0.0, 'cross_l2': 0.01, 'cross_l1': 0.0, 'reg_kg': 0.0, 'learning_rate': 0.002, 'lr_rs': 1, 'lr_kg': 0.5, 'kg_training_interval': 5, 'max_grad_norm': 2, 'is_clip_norm': 0, 'dtype': 32, 'optimizer': 'adam', 'epochs': 10, 'batch_size': 4096, 'enable_BN': False, 'show_step': 200000, 'save_model': False, 'save_epoch': 2, 'write_tfevents': False, 'train_num_ngs': 4, 'need_sample': True, 'embedding_dropout': 0.0, 'EARLY_STOP': 100, 'min_seq_length': 1, 'slots': 5, 'cell': 'SUM', 'FIELD_COUNT': 39, 'FEATURE_COUNT': 2300000, 'data_format': 'ffm', 'load_model_n

In [9]:
model = XDeepFMModel(hparams, FFMTextIterator, seed=RANDOM_SEED)

Add linear part.
Add CIN part.
Add DNN part.


2024-03-09 11:12:30.925035: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-09 11:12:30.967250: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-03-09 11:12:31.021175: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


In [10]:
# check the predictive performance before the model is trained
print(model.run_eval(test_file)) 

{'auc': 0.4728, 'logloss': 0.7113}


In [11]:
train_file

'/tmp/tmpo55qjar0/cretio_tiny_train'

In [12]:
model.fit(train_file, valid_file)

at epoch 1
train info: logloss loss:744.3601875305176
eval info: auc:0.6637, logloss:0.5342
at epoch 1 , train time: 11.8 eval time: 1.8
at epoch 2
train info: logloss loss:385.6692638397217
eval info: auc:0.7137, logloss:0.5109
at epoch 2 , train time: 11.6 eval time: 1.7
at epoch 3
train info: logloss loss:191.50831031799316
eval info: auc:0.7283, logloss:0.5037
at epoch 3 , train time: 11.5 eval time: 1.8
at epoch 4
train info: logloss loss:92.20773839950562
eval info: auc:0.7359, logloss:0.4991
at epoch 4 , train time: 11.5 eval time: 1.8
at epoch 5
train info: logloss loss:43.15945327281952
eval info: auc:0.74, logloss:0.4963
at epoch 5 , train time: 11.5 eval time: 1.8
at epoch 6
train info: logloss loss:19.656920433044434
eval info: auc:0.7426, logloss:0.4946
at epoch 6 , train time: 11.6 eval time: 1.7
at epoch 7
train info: logloss loss:8.770356237888336
eval info: auc:0.7441, logloss:0.4934
at epoch 7 , train time: 11.6 eval time: 1.7
at epoch 8
train info: logloss loss:3.922

In [13]:
# check the predictive performance after the model is trained
result = model.run_eval(test_file)
print(result)

{'auc': 0.7356, 'logloss': 0.5017}


In [14]:
?model.fit

Signature: model.fit(train_file, valid_file, test_file=None)
Docstring:
Fit the model with `train_file`. Evaluate the model on valid_file per epoch to observe the training status.
If `test_file` is not None, evaluate it too.

Args:
    train_file (str): training data set.
    valid_file (str): validation set.
    test_file (str): test set.

Returns:
    object: An instance of self.
File:      /opt/conda/envs/rec/lib/python3.9/site-packages/recommenders/models/deeprec/models/base_model.py
Type:      method

In [15]:
!ls train_file

'/tmp/tmpo55qjar0/cretio_tiny_train'

In [18]:
with open("/tmp/tmpo55qjar0/cretio_tiny_train") as f:
    s = f.readlines()

In [20]:
s[:1]

['0 16:372270:1 11:1561051:1 24:199173:1 3:1750186:1 26:2073461:1 12:1864800:1 17:150363:1 18:1283644:1 28:164307:1 27:1262273:1 19:1349205:1 7:1928194:1 39:795398:1 9:161196:1 32:1480661:1 30:180793:1 10:267940:1 1:1533602:1 21:1829128:1 20:602253:1 22:1113685:1 8:587827:1 29:1978347:1 4:1007621:1 35:635625:1 14:1712964:1 36:505691:1 2:1767161:1 25:94115:1 37:1331841:1 5:1140056:1 15:1080229:1 38:1372774:1 23:1055765:1 34:265347:1 31:980768:1 33:1483574:1 13:108951:1 6:904241:1\n']

In [21]:
model.

<tf.Variable 'XDeepFM/embedding/embedding_layer:0' shape=(2300000, 10) dtype=float32>